In [29]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [30]:
train_data = pd.read_csv('datass/Files/train_emoji.csv')
test_data = pd.read_csv('datass/Files/tesss.csv')

In [31]:
X_train = train_data['never talk to me again']
y_train = train_data['3']
X_train[131] = 'never talk to me again'
y_train[131] = 3

/tmp/ipykernel_225827/3497168708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[131] = 'never talk to me again'
/tmp/ipykernel_225827/3497168708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train[131] = 3


In [32]:
X_test = test_data['I want to eat\t'].map(lambda x:x.replace("\t",""))
y_test = test_data['4']

In [33]:
def convert_to_one_hot(y_train , depth = 5):
    one_hot_ds = []
    for i in range(len(y_train)):
        temp = [0]*depth
        temp[y_train[i]]+=1
        one_hot_ds.append(temp)
    np_array = np.array(one_hot_ds)
    return np_array

In [34]:
with open("datass/Files/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    word_to_vec_map = {}
    word_to_index_map = {}
    index_to_word_map = {}
    index = 1
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_to_vec_map[word] = vector
        word_to_index_map[word] = index
        index_to_word_map[index] = word
        index+=1

In [35]:
# def sentence_to_indices(sentence):
#     sentenceArr = sentence.lower().split()
#     sentenceArr = [word_to_index_map[x] for x in sentenceArr]
#     return sentenceArr

In [162]:
def sentence_to_indices(sentence,maxLength = 45):
    m = sentence.shape[0]
    finalArr = np.zeros((m,maxLength),dtype = 'int')
    for i in range(m):
        ind = 0
        for j in sentence[i].lower().split():
            if j in word_to_index_map.keys():
                finalArr[i][ind] = word_to_index_map[j]
                ind+=1
    return finalArr

In [37]:
sentArr = pd.array(["hello","how are you","wassup my boy"])
sentence_to_indices(sentArr)

array([[ 13076,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [   198,     33,     82,      0,      0,      0,      0,      0,
             0,      0],
       [243124,    193,   1607,      0,      0,      0,      0,      0,
             0,      0]])

In [38]:
def pretrained_embedding_layer(word_to_vec_map , index_to_word_map):
    vocab_size = len(word_to_vec_map.keys()) + 1
    random_word = word_to_vec_map['random']
    
    emb_matrix = np.zeros((vocab_size , random_word.shape[0]))
    for idx,word in index_to_word_map.items():
        emb_matrix[idx,:] = word_to_vec_map[word]
    
    embedding_layer = tf.keras.layers.Embedding(vocab_size , random_word.shape[0],trainable=False)
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [39]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map , index_to_word_map)

In [200]:
def EmojifyV2():
    sentence_indices = tf.keras.layers.Input(shape = (45,),dtype = 'int32')
    embeddings = embedding_layer(sentence_indices)
    X = tf.keras.layers.LSTM(units = 200 , return_sequences = True)(embeddings)
    X = tf.keras.layers.Dropout(rate = 0.5)(X)
    X = tf.keras.layers.LSTM(units = 200,return_sequences = True)(embeddings)
    X = tf.keras.layers.Dropout(rate = 0.5)(X)
    X = tf.keras.layers.LSTM(units = 200)(embeddings)
    X = tf.keras.layers.Dropout(rate = 0.5)(X)
    X = tf.keras.layers.Dense(units = 20)(X)
    X = tf.keras.layers.Activation('softmax')(X)
    model = tf.keras.Model(inputs = sentence_indices , outputs = X)
    return model

In [201]:
model =  EmojifyV2()
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 45)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 45, 50)         │    20,000,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 200)            │       200,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 20)             │         4,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 20)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,204,870 (77.08 MB)

 Trainable params: 204,820 (800.08 KB)

 Non-trainable params: 20,000,050 (76.29 MB)

In [202]:
model.compile(loss = "categorical_crossentropy"  ,optimizer='adam', metrics = ["accuracy"])

In [94]:

from datasets import load_dataset
ds = load_dataset("cardiffnlp/tweet_eval", "emoji")
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [120]:
tf_ds_train = ds['train'].to_tf_dataset()
tf_ds_test = ds['test'].to_tf_dataset()
tf_ds_valid = ds['validation'].to_tf_dataset()

In [108]:
for x in tf_ds_train.take(1):
    print(x['text'].numpy())

b'Sunday afternoon walking through Venice in the sun with @user \xef\xb8\x8f \xef\xb8\x8f \xef\xb8\x8f @ Abbot Kinney, Venice'


In [109]:
maxLength = 0
for x in tf_ds_train:
    maxLength = max(maxLength,len(x['text'].numpy().split()))
print(maxLength)

39


2025-08-18 00:18:52.139968: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [110]:
maxy = 0
for x in tf_ds_train:
    # print(tf.one_hot(x['label'],depth = 10))
    maxy = max(maxy,x['label'].numpy())
print(maxy)

19


In [131]:
lookupTable = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(list(word_to_index_map.keys()),list(word_to_index_map.values())),
    default_value = -1
)
def preprocess_data(example,max_len = 45,pad_value = 0):
    example['text'] = tf.strings.lower(input = example['text'])
    example['text'] = tf.strings.split(input = example['text'])
    example['text'] = tf.ragged.map_flat_values(
        lookupTable.lookup,
        example['text']
    )
    example['text'] = example['text'][:max_len]
    
    # Pad if shorter than max_len
    current_len = tf.shape(example['text'])[0]
    pad_len = max_len - current_len
    example['text'] = tf.pad(example['text'], [[0, pad_len]], constant_values=pad_value)
    example['label']  = tf.one_hot(example['label'],depth = 20)
    return (example['text'] , example['label'])
for x,y in tf_ds_train.take(1).map(preprocess_data):
    print(x)
# tf_ds_train.take(1).map(preprocess_data)

tf.Tensor(
[  241  1503  3527   132  8082     7     1  1663    18    -1    -1    -1
    -1 17528 16326    -1  8082     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0], shape=(45,), dtype=int32)


In [136]:
for x in tf_ds_train.take(1):
    print(x)

(<tf.Tensor: shape=(34, 45), dtype=int32, numpy=
array([[  241,  1503,  3527, ...,     0,     0,     0],
       [   80,    11,    78, ...,     0,     0,     0],
       [  836,   836,   836, ...,     0,     0,     0],
       ...,
       [   -1,    -1,   255, ...,     0,     0,     0],
       [   -1,     5,     1, ...,     0,     0,     0],
       [  589, 10231,  1579, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(34, 20), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [203]:
model.fit(tf_ds_train , epochs = 50, batch_size = 34)

Epoch 1/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 31s 22ms/step - accuracy: 0.1970 - loss: 2.7675
Epoch 2/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 30s 22ms/step - accuracy: 0.2131 - loss: 2.6953
Epoch 3/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 29s 22ms/step - accuracy: 0.2376 - loss: 2.6380
Epoch 4/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 30s 22ms/step - accuracy: 0.2636 - loss: 2.5417
Epoch 5/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 30s 23ms/step - accuracy: 0.2760 - loss: 2.4784
Epoch 6/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 30s 23ms/step - accuracy: 0.2857 - loss: 2.4414
Epoch 7/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 30s 23ms/step - accuracy: 0.2941 - loss: 2.4108
Epoch 8/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 31s 23ms/step - accuracy: 0.3038 - loss: 2.3786
Epoch 9/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 30s 23ms/step - accuracy: 0.3140 - loss: 2.3448
Epoch 10/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 31s 24ms/step - accuracy: 0.3276 - loss: 2.3061
Epoch 11/50
1324/1324 ━━━━━━━━━━━━━━━━━━━━ 31s 23ms/step - accuracy: 0.3370 - loss: 2.2677
Epoch 12

In [219]:
model.save('my_model.keras')

In [221]:
model.evaluate(tf_ds_valid)

ValueError: Missing data for input "input_layer_5". You passed a data dictionary with keys ['text', 'label']. Expected the following keys: ['input_layer_5']

In [205]:
label_to_emoji = {
    0: "❤",
    1: "😍",
    2: "😂",
    3: "💕",
    4: "🔥",
    5: "😊",
    6: "😎",
    7: "✨",
    8: "💙",
    9: "😘",
    10: "📷",
    11: "🇺🇸",
    12: "☀",
    13: "💜",
    14: "😉",
    15: "💯",
    16: "😁",
    17: "🎄",
    18: "📸",
    19: "😜"
}


In [206]:
def predict_emoji(predArr,predictions):
    for i in range(len(predictions)):
        print(f"{predArr[i]} {label_to_emoji[np.argmax(predictions[i])]}")

In [216]:
origArr = ["I hate you","Click a picture with me"]
predArr = sentence_to_indices(np.array(origArr))

In [217]:
predictions = model.predict(predArr)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [218]:
predict_emoji(origArr,predictions)

I hate you 😊
Click a picture with me 🔥
